In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import qiskit
from qiskit import QuantumCircuit, Aer, transpile, IBMQ, execute, schedule
from qiskit.test.mock.backends.toronto.fake_toronto import FakeToronto
from qiskit.test.mock.backends.rome.fake_rome import FakeRome

In [ ]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-sherbrooke')

In [ ]:
qasm_sim = provider.get_backend('ibmq_qasm_simulator')
sydney = provider.get_backend('ibmq_sydney')

In [ ]:
syd_conf = sydney.configuration()
syd_prop = sydney.properties()

## One quibit circuit

In [ ]:
def x_measure():
    qc = QuantumCircuit(1,1)
    qc.h(0)
    qc.measure()
    qc.h(0)
    return qc
xmeas = x_measure()
xmeas_native = transpile(qc, sydney)
xmeas_sched = schedule(xmeas_native, sydney)

In [ ]:
# Find highest single-qubit gate error rate
highest = 0
qubit = None
for q in range(syd_conf.num_qubits):
    current = syd_prop.gate_error('sx', q)
    if highest < current:
        qubit = q
        highest = current
print(f'Highest single-qubit gate error rate: qubit {qubit}')

In [ ]:
slots = 3
circuits = []

meas_circ = QuantumCircuit(1,1)
meas_circ.measure(0,0)
meas_sched = schedule(meas_circ, sydney)

for i in range(slots):
    qc = QuantumCircuit(1,1)
    qc.h(0)
    sched = schedule(transpile(qc, sydney), sydney)
    for _ in range(i):
        sched = sched.append(xmeas_sched)
    delayed_meas = meas_sched.shift((slots-i-1) * xmeas_sched.duration)
    sched = sched.append(delayed_meas)
    circuits.append(sched)

In [ ]:
job = execute(circuits, sydney, initial_layout=[qubit]).result()

In [ ]:
syd_conf.to_dict()['qubit_lo_range']